# Initialisation
Select the mode in `pandas`, `cudf`, `dask` or `dask-cudf`.

In [ ]:
#%reload_ext autoreload
#%autoreload 2
import os
os.environ["VDF_MODE"] = "dask"  # Reset the kernel if you change this

#import cardif_dask as vdf  # Import Virtual Dataframe
import virtual_dataframe as vdf
import pandas as pd
import cupy as cp

print(f"Use {vdf.VDF_MODE}")

# Object Creation
Creating a `VSeries`

In [ ]:
s = vdf.VSeries([1,2,3,None,4],npartitions=2)
s.compute()

Creating a `VDataFrame` by specifying values for each column.

In [ ]:
df = vdf.VDataFrame({'a': list(range(20)),
                 'b': list(reversed(range(20))),
                 'c': list(range(20))
                }, npartitions=2)
df.compute()

Creating a `VDataFrame` from a pandas `Dataframe`.

> Note that best practice for `VDataFrame` is to read data directly into a ̀`VDataFrame` with something like `read_csv()` (discussed below).

In [ ]:
pdf = pd.DataFrame({'a': [0, 1, 2, 3],'b': [0.1, 0.2, 0.3, 0.4]})
df = vdf.from_pandas(pdf,npartitions=2)
df.compute()

In [ ]:
ps = pd.Series([1,2,3,None,4])
s = vdf.from_pandas(ps,npartitions=2)
s.compute()

# Viewing Data
Viewing the top rows of a `VDataFrame`.

In [ ]:
df.head(2).compute()

In [ ]:
df.sort_values(by='b').compute()

# Selection
## Getting
Selecting a single column, which initially yields a `VSeries`.

In [ ]:
df['a'].compute()

## Selection by Label
Selecting rows from index 2 to index 5 from columns ‘a’ and ‘b’.

In [ ]:
df.loc[2:5, ['a', 'b']]

In [ ]:
df.loc[2:5, ['a', 'b']].compute()

## Selection by Position
Selecting via integers and integer slices, like numpy/pandas.
> Note that this functionality is not available for `dask-cudf`.

In [ ]:
df.iloc[0].compute()

In [ ]:
df.iloc[0].compute() if vdf.VDF_MODE != vdf.Mode.dask_cudf else None

In [ ]:
df.iloc[0:3, 0:2].compute()

You can also select elements of a `VDataFrame` or `VSeries` with direct index access.

In [ ]:
df[3:5].compute()

In [ ]:
s[3:5].compute()

## Boolean Indexing
Selecting rows in a `VDataFrame` or `VSeries` by direct `Boolean` indexing.

In [ ]:
df[df.b > 15].compute()

Selecting values from a `DataFrame` where a `Boolean` condition is met, via the query API.

In [ ]:
df.query("b == 3").compute()

You can also pass local variables to Dask-cuDF queries, via the local_dict keyword.
With standard cuDF, you may either use the local_dict keyword or directly pass the variable
via the @ keyword. Supported logical operators include >, <, >=, <=, ==, and !=.

In [ ]:
value = 3
df.query("b == @value").compute()

In [ ]:
value = 3
df.query("b == @val", local_dict={'val':value}).compute()

Using the isin method for filtering.

In [ ]:
df[df.a.isin([0, 5])].compute()

# Missing Data
Missing data can be replaced by using the fillna method.

In [ ]:
s.fillna(999).compute()

# Operations
## Stats
Calculating descriptive statistics for a Series.

In [ ]:
vdf.compute(s.mean())[0], vdf.compute(s.var())[0]

# Applymap
Applying functions to a Series.
Note that applying user defined functions directly with Dask-cuDF is not yet implemented.
For now, you can use map_partitions to apply a function to each partition of the distributed dataframe.

In [ ]:
def add_ten(num):
    return num + 10

df['a'].apply(add_ten).compute()

In [ ]:
# Not with pandas
if vdf.VDF_MODE in (vdf.Mode.dask, vdf.Mode.dask_cudf):
    df['a'].map_partitions(add_ten).compute()

# Histogramming
Counting the number of occurrences of each unique value of variable.

In [ ]:
df.a.value_counts().compute()

# String Methods
Virtual Dataframe provides string processing methods in the str attribute of Series.

In [ ]:
s = vdf.VSeries(['A', 'B', 'C', 'Aaba', 'Baca', None, 'CABA', 'dog', 'cat'], npartitions=2)
s.str.lower().compute()

# Concat
Concatenating VSeries and VDataFrames row-wise.

In [ ]:
s = vdf.VSeries([1, 2, 3, None, 5],npartitions=2)
vdf.concat([s, s]).compute()

# Join
Performing SQL style merges.
Note that the dataframe order is not maintained, but may be restored post-merge by sorting by the index.

In [ ]:
df_a = vdf.VDataFrame()
df_a['key'] = ['a', 'b', 'c', 'd', 'e']
df_a['vals_a'] = [float(i + 10) for i in range(5)]

df_b = vdf.VDataFrame()
df_b['key'] = ['a', 'c', 'e']
df_b['vals_b'] = [float(i+100) for i in range(3)]

merged = df_a.merge(df_b, on=['key'], how='left').compute()
merged

# Grouping
Virtual Dataframe support the Split-Apply-Combine groupby paradigm.

In [ ]:
import cudf
import dask_cudf
df = vdf.VDataFrame({'a': list(range(20)),
                     'b': list(reversed(range(20))),
                     'c': list(range(20))
                    })
df.compute()

df['agg_col1'] = pd.Series([1 if x % 2 == 0 else 0 for x in range(len(df))])
df['agg_col2'] = pd.Series([1 if x % 3 == 0 else 0 for x in range(len(df))])

ddf = vdf.from_virtual(df.compute(), npartitions=2)
ddf.compute()

Grouping and then applying the sum function to the grouped data.

In [ ]:
df.groupby('agg_col1').sum().compute()

Grouping hierarchically then applying the sum function to grouped data.

In [ ]:
df.groupby(['agg_col1', 'agg_col2']).sum().compute()

Grouping and applying statistical functions to specific columns, using agg.

In [ ]:
df.groupby('agg_col1').agg({'a':'max', 'b':'mean', 'c':'sum'}).compute()

# FIXME Time Series
`VDataFrames` supports datetime typed columns, which allow users to interact with and filter data based on specific timestamps.

In [ ]:
import datetime as dt
import pandas as pd
pd.Series(pd.date_range('11/20/2018', periods=72, freq='D'))

In [ ]:
#date_df = vdf.VDataFrame()
#date_df['date'] = pd.date_range('11/20/2018', periods=72, freq='D')
import dask
import pandas
pdf=pandas.DataFrame(index=pd.Series(pd.date_range('11/20/2022', periods=72, freq='D')))
pdf

In [ ]:
date_df = dask.dataframe.from_pandas(pdf,npartitions=2)
date_df
#date_df.set_index("date",pd.Series(pd.date_range('11/20/2018', periods=72, freq='D')))

In [ ]:
import datetime as dt

date_values=pd.Series(pd.date_range('11/20/2022', periods=72, freq='D'))
data_values=rng.standard_normal(len(date_values))
vdf.VDataFrame(data_values,index=date_values,npartitions=2)

In [ ]:
from numpy.random import default_rng
rng = default_rng()
vals = rng.standard_normal(10)
more_vals = rng.standard_normal(10)
rng, vals, more_vals

In [ ]:
# FIXME
from numpy.random import default_rng
rng=default_rng()
rand_len = cp.random.sample(len(date_df))

# FIXME Categoricals
VDataFrames support categorical columns.

In [ ]:
# FIXME
cdf = vdf.VDataFrame({"id": [1, 2, 3, 4, 5, 6], "grade":['a', 'b', 'b', 'a', 'a', 'e']}, npartitions=2)
cdf['grade'] = df['grade'].astype('category')
cdf.compute()

Accessing the categories of a column.

> Note that this is currently not supported in `dask-cudf`.

In [ ]:
# FIXME
cdf.grade.cat.categories

Accessing the underlying code values of each categorical observation.

In [ ]:
cdf.grade.cat.codes.compute()

# Converting Data Representation
## Pandas
Converting a `VDataFrame` to a pandas DataFrame.

In [ ]:
df.head().to_pandas()

## Numpy
Converting a `VDataFrame` to a numpy ndarray.

In [ ]:
df.to_numpy()

Converting a `VSeries` to a numpy ndarray.

In [ ]:
df['a'].to_numpy()

## Arrow
Converting a `VDataFrame` to a PyArrow Table.

> Note that this functionality is not available for `pandas` or `dask`.

In [ ]:
if vdf.VDF_MODE in (vdf.Mode.cudf, vdf.Mode.dask_cudf):
    df.to_arrow()

# Getting Data In/Out
## CSV
Writing to a CSV file.

In [ ]:
if not os.path.exists('example_output'):
    os.mkdir('example_output')

df.to_csv('example_output/foo.csv', index=False)

Reading from a csv file.

In [ ]:
df = vdf.read_csv('example_output/foo.csv')
df.compute()

Reading all CSV files in a directory into a single dask_cudf.DataFrame, using the star wildcard.
> Not implemented in pandas or cudf ?

In [ ]:
# FIXME: extends pandas
if vdf.VDF_MODE in (vdf.Mode.dask, vdf.Mode.dask_cudf):
    df = vdf.read_csv('example_output/*.csv')
    df.compute()

# Parquet
Writing to parquet files, using the CPU via PyArrow.

In [ ]:
df.to_parquet('example_output/temp_parquet')

## ORC
Reading ORC files.
> FIXME

In [ ]:
if vdf.VDF_MODE in (vdf.Mode.dask, vdf.Mode.dask_cudf):
    from pathlib import Path
    cudf_root = Path(".").absolute().parents[3]
    orc_file = Path("python/cudf/cudf/tests/data/orc/TestOrcFile.test1.orc")
    file_path = cudf_root / orc_file

In [ ]:
# FIXME
df2 = vdf.read_orc(file_path)
df2